## All reviews from all games

In [1]:
import sys
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import input_file_name
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window


sys.executable

'/bin/python3'

In [2]:
CREDENTIALS = '/home/vyago/.google/credentials/google_credentials.json'
GCS_JAR = "./lib/gcs-connector-hadoop3-latest.jar"
BQ_JAR = "./lib/spark-3.1-bigquery-0.27.0-preview.jar"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "{GCS_JAR}, {BQ_JAR}") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", CREDENTIALS)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

22/09/24 01:33:00 WARN Utils: Your hostname, vyago-desktop resolves to a loopback address: 127.0.1.1; using 192.168.18.7 instead (on interface enp4s0)
22/09/24 01:33:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/24 01:33:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/24 01:33:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

Connect to GCS

In [5]:
if False : 
    bucket_name="steam-datalake-reviews"
    path=f"gs://steam-datalake-reviews/raw/10/416F4A342B2B663375766F43633454673667493D.json"
    df=spark.read.json(path)
    df.show()

In [6]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df


def proc_json(raw_df, field)  :
    rows = raw_df[field]
    return(rows)

Dir with a sample of the games :

In [7]:
reviews_lite_dir = "/home/vyago/Documents/steam-data-engineering/reviews_lite/*/*"

In [8]:
all_games = spark.read.json(reviews_lite_dir).withColumn("filename", input_file_name())
all_games.printSchema()
all_games.show()

root
 |-- cursor: string (nullable = true)
 |-- query_summary: struct (nullable = true)
 |    |-- num_reviews: long (nullable = true)
 |    |-- review_score: long (nullable = true)
 |    |-- review_score_desc: string (nullable = true)
 |    |-- total_negative: long (nullable = true)
 |    |-- total_positive: long (nullable = true)
 |    |-- total_reviews: long (nullable = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author: struct (nullable = true)
 |    |    |    |-- last_played: long (nullable = true)
 |    |    |    |-- num_games_owned: long (nullable = true)
 |    |    |    |-- num_reviews: long (nullable = true)
 |    |    |    |-- playtime_at_review: long (nullable = true)
 |    |    |    |-- playtime_forever: long (nullable = true)
 |    |    |    |-- playtime_last_two_weeks: long (nullable = true)
 |    |    |    |-- steamid: string (nullable = true)
 |    |    |-- comment_count: long (nullable = true)
 |    |    |--

In [9]:
rdd = all_games.rdd
print(f"Number of files processed: {rdd.count()}")

Number of files processed: 97


### Reviews 

In [10]:
df_reviews = rdd.flatMap(lambda item : proc_json(item, field = "reviews")).toDF()
df_reviews.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- last_played: long (nullable = true)
 |    |-- num_games_owned: long (nullable = true)
 |    |-- num_reviews: long (nullable = true)
 |    |-- playtime_at_review: long (nullable = true)
 |    |-- playtime_forever: long (nullable = true)
 |    |-- playtime_last_two_weeks: long (nullable = true)
 |    |-- steamid: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)



### Game ID 

In [11]:
df_gameid = rdd.map(  lambda item : [ proc_json(item, field = "reviews"),
                                          proc_json(item, field = "filename")] ) \
                   .flatMap(lambda item:  [item[1] for i in item[0] ]) \
                   .map(lambda item : Row(gameid  = item, )).toDF()
                   
# TODO EXTRACT GAME ID FROM DIR


df_gameid.printSchema()

root
 |-- gameid: string (nullable = true)



### Join dataframes

In [12]:
w = Window().orderBy(lit('A'))
df_reviews = df_reviews.withColumn("row_num", row_number().over(w))
df_gameid = df_gameid.withColumn("row_num", row_number().over(w))

df_reviews = df_reviews.join(df_gameid, on = ["row_num"], how = "inner")

In [13]:
df_reviews.printSchema()

root
 |-- row_num: integer (nullable = true)
 |-- author: struct (nullable = true)
 |    |-- last_played: long (nullable = true)
 |    |-- num_games_owned: long (nullable = true)
 |    |-- num_reviews: long (nullable = true)
 |    |-- playtime_at_review: long (nullable = true)
 |    |-- playtime_forever: long (nullable = true)
 |    |-- playtime_last_two_weeks: long (nullable = true)
 |    |-- steamid: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: bool

In [14]:
df_reviews.select("gameid").show()

22/09/24 01:33:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/24 01:33:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+
|              gameid|
+--------------------+
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
|file:/home/vyago/...|
+--------------------+
only showing top 20 rows



In [15]:
df_reviews.count()

22/09/24 01:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/24 01:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


9300

In [16]:
df_reviews.take(1)

22/09/24 01:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/24 01:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


[Row(row_num=1, author=Row(last_played=1656773225, num_games_owned=102, num_reviews=53, playtime_at_review=65, playtime_forever=85, playtime_last_two_weeks=0, steamid='76561198141524029'), comment_count=0, language='koreana', received_for_free=False, recommendationid='64270873', review='[ 희망 가격 ]\n□ 정가\n□ 20% 할인\n□ 50% 할인\n■ 75% 이상\n\n[ 그래픽 ]\n★★★☆☆\n\n그래픽은 당시 기준으로 좋은 편이나, 팀 포트리스2에 비해서 아트 디자인의 특색이 없는 것같아 별점 3점입니다.\n\n[ 사운드 ]\n★★★★☆\n\n사운드 디자인 또한 팀포트리스2의 상징적일 정도의 사운드 디자인에 조금 비교가 되지 않나 생각되지만 나름의 매력이 있습니다.\n\n[ 난이도 ]\n□ 쉬움\n□ 중간\n■ 어려움\n\n한국 서버가 존재하지 않아 핑이 매우 높기 때문에 정상적인 플레이가 불가능합니다. 따라서 적을 처치하는 경우는 드물고 죽는 경우가 매우 많습니다.\n\n[ 버그 ]\n■ 없음\n□ 낮은 빈도\n□ 높은 빈도\n\n[ 게임성 ]\n★★★★☆\n\n높은 핑으로 인해 정상적인 플레이는 불가해서 정확한 판단을 할 수는 없었지만 그래도 낮은 핑의 서버가 있다면 매우 재밌게 즐길 수있을 것같습니다.\n\n----\n\n골드 소스 시절 밸브 온라인 게임 중에서는 의외로 사람이 많은 편에 속하네요. 시간대에 따라 달라지겠지만 접속해보면 보통 최대 32명을 수용할 수있는 서버 2~3곳 정도는 꽉차있습니다.', steam_purchase=True, timestamp_created=1582940659, timestamp_updated=1583905902, voted_up=True, votes_funny=0, votes_up=5,

In [17]:
df_reviews_flat = flatten_df(df_reviews) 
df_reviews_flat.printSchema()

root
 |-- row_num: integer (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- language: string (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- recommendationid: string (nullable = true)
 |-- review: string (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- voted_up: boolean (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- votes_up: long (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)
 |-- gameid: string (nullable = true)
 |-- author_last_played: long (nullable = true)
 |-- author_num_games_owned: long (nullable = true)
 |-- author_num_reviews: long (nullable = true)
 |-- author_playtime_at_review: long (nullable = true)
 |-- author_playtime_forever: long (nullable = true)
 |-- author_playtime_last_two_weeks: long (nullable = true)
 |-- autho

### Write into BigQuery

- https://github.com/GoogleCloudDataproc/spark-bigquery-connector

In [18]:
# Saving the data to BigQuery
df_reviews_flat.write \
  .format('bigquery') \
  .option('table', 'steam-data-engineering-gcp.steam_raw.reviews') \
  .mode("append") \
  .option("temporaryGcsBucket","steam-datalake-reviews") \
  .save()

22/09/24 01:33:13 WARN DefaultCredentialsProvider: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
22/09/24 01:33:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/24 01:33:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
